In [1]:
text_list = []
with open("../src/shevchenko/taras_shevchenko.txt", 'r', encoding='utf8') as sh_file:
    i = 0
    for line in sh_file:
        if line == '\n':
            continue
        text_list.append(line)

print('text_list len:', len(text_list))

for line in text_list:
    print(line)

text_list len: 12348
Том 2

ТАРАС ШЕВЧЕНКО

ПОЕТИЧНІ ТВОРИ

1847–1861

В КАЗЕМАТІ

Моїм соузникам посвящаю

Згадайте, братія моя…

Згадайте, братія моя…

Бодай те лихо не верталось,

Як ви гарнесенько і я

Із-за решотки визирали.

І, певне, думали: «Коли

На раду тиху, на розмову,

Коли ми зійдемося знову

На сій зубоженій землі?»

Ніколи, братія, ніколи

З Дніпра укупі не п’ємо!

Розійдемось, рознесемо

В степи, в ліси свою недолю,

Повіруєм ще трохи в волю,

А потім жити почнемо

Меж людьми, як люде.

А поки те буде,

Любітеся, брати мої,

Украйну любіте

І за неї, безталанну,

Господа моліте.

І його забудьте, други,

І не проклинайте.

І мене в неволі лютій

Інколи згадайте.

[1847, Орська кріпость]

I. «Ой одна я, одна…»

Ой одна я, одна,

Як билиночка в полі,

Та не дав мені бог

Ані щастя, ні долі.

Тілько дав мені бог

Красу — карії очі,

Та й ті виплакала

В самотині дівочій.

Ані братика я,

Ні сестрички не знала,

Меж чужими зросла,

І зросла — не кохалась!

Де ж дружина моя

In [2]:
text = ''

for txt in text_list:
    text += txt.lower()

chars = sorted(list(set(text)))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

print('Corpus length: {}\nTotal chars: {}\n'.format(len(text), len(chars)))

Corpus length: 299945
Total chars: 78



In [3]:
print(text[:500])

том 2
тарас шевченко
поетичні твори
1847–1861
в казематі
моїм соузникам посвящаю
згадайте, братія моя…
згадайте, братія моя…
бодай те лихо не верталось,
як ви гарнесенько і я
із-за решотки визирали.
і, певне, думали: «коли
на раду тиху, на розмову,
коли ми зійдемося знову
на сій зубоженій землі?»
ніколи, братія, ніколи
з дніпра укупі не п’ємо!
розійдемось, рознесемо
в степи, в ліси свою недолю,
повіруєм ще трохи в волю,
а потім жити почнемо
меж людьми, як люде.
а поки те буде,
любітеся, брати мо


In [4]:
maxlen = 40
step = 3
sentences = []
next_chars = []

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i:i + maxlen])
    next_chars.append(text[i + maxlen])

print('nb sequences:', len(sentences))

nb sequences: 99969


In [5]:
import numpy as np

X = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool_)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool_)

for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

## Assemble a char-based LSTM model for generating text

In [6]:
from keras.models import Sequential
from keras.layers import Dense, Activation, LSTM
from keras.optimizers import RMSprop


model = Sequential()
model.add(LSTM(units=196, input_shape=(maxlen, len(chars))))
model.add(Dense(units=len(chars)))
model.add(Activation(activation='softmax'))

optimizer = RMSprop(learning_rate=.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 196)               215600    
                                                                 
 dense (Dense)               (None, 78)                15366     
                                                                 
 activation (Activation)     (None, 78)                0         
                                                                 
Total params: 230,966
Trainable params: 230,966
Non-trainable params: 0
_________________________________________________________________


In [7]:
epochs = 10
batch_size = 128

model_structure = model.to_json()
with open("../src/part_8/char_gen_lstm_shevchenko/shev_lstm_model.json", 'w') as json_file:
    json_file.write(model_structure)

for i in range(5):
    model.fit(X, y,
              batch_size=batch_size,
              epochs=epochs)
    model.save_weights('../src/part_8/char_gen_lstm_shevchenko/shev_lstm_weights_{}.h5'.format(i + 1))

Epoch 1/10
782/782 [==============================] - 8s 6ms/step - loss: 2.6324
Epoch 2/10
782/782 [==============================] - 5s 6ms/step - loss: 2.1384
Epoch 3/10
782/782 [==============================] - 5s 6ms/step - loss: 1.9585
Epoch 4/10
782/782 [==============================] - 5s 6ms/step - loss: 1.8458
Epoch 5/10
782/782 [==============================] - 5s 7ms/step - loss: 1.7617
Epoch 6/10
782/782 [==============================] - 5s 6ms/step - loss: 1.6965
Epoch 7/10
782/782 [==============================] - 5s 6ms/step - loss: 1.6409
Epoch 8/10
782/782 [==============================] - 5s 6ms/step - loss: 1.5975
Epoch 9/10
782/782 [==============================] - 5s 6ms/step - loss: 1.5622
Epoch 10/10
782/782 [==============================] - 5s 6ms/step - loss: 1.5355
Epoch 1/10
782/782 [==============================] - 5s 6ms/step - loss: 1.5037
Epoch 2/10
782/782 [==============================] - 5s 6ms/step - loss: 1.4764
Epoch 3/10
782/782 [=======

In [8]:
import random


def sample(preds, temperature=1.):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [9]:
import sys

def text_gen(diversities=[.2]):
    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in diversities:
        print('\n----- diversity:', diversity)
        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generaring with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x = np.zeros((1, maxlen, len(chars)))
            for t, char  in enumerate(sentence):
                x[0, t, char_indices[char]] = 1.
            preds = model.predict(x, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]
            generated += next_char
            sentence = sentence[1:] + next_char
            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

In [10]:
text_gen(diversities=[.2])


----- diversity: 0.2
----- Generaring with seed: "истая в женах!
благоуханний сельний крин"
истая в женах!
благоуханний сельний криниці помарів.
13
дальським давно озновов катерини монова
на світі про

C:\Users\voldo\AppData\Local\Temp\ipykernel_8036\284865468.py:6: RuntimeWarning: divide by zero encountered in log
  preds = np.log(preds) / temperature


клять і надівать,
щоб замінку за сонце вар
на світі веселить,
та й прокляту! а я таки йстра
на світі на світі на світі,
на тій самотури, і в окупом.
158
спербуш — собі, запорожнна (вле шевченко не вар. один ж новий, за полтав поверіли; вчан» що ви з калекії петербурязи полетась)
«дівчина виростала,
в садочку понеєли,
і в оклада за


In [11]:
text_gen(diversities=[.5])


----- diversity: 0.5
----- Generaring with seed: "гуляєм понад морем
удвох собі. дивлюся, "
гуляєм понад морем
удвох собі. дивлюся, як тихо
в холодочку волю

C:\Users\voldo\AppData\Local\Temp\ipykernel_8036\284865468.py:6: RuntimeWarning: divide by zero encountered in log
  preds = np.log(preds) / temperature


 цигай,
та казав батько, мов те, — каже
і на роки чорнобровиця
в слова вода,
карита в полетася,
і справді із дитини,
на всій україні, куморала
і за дворах хати,
щоб здав!. не проблага,
та й спита на когонь наливає
і бачить розмовляв і наглагатим,
мов продав боже ми остала
і в світі україни
і старий віна божі поко!
мене та й покрилась,
та й забідуть.
а до трухи положили
на с


In [12]:
text_gen(diversities=[1.])


----- diversity: 1.0
----- Generaring with seed: "зично гукає:
«нехай вам, панове товарист"
зично гукає:
«нехай вам, панове товариство,
яке зо горою.
. iii
ії
у когонь нево

C:\Users\voldo\AppData\Local\Temp\ipykernel_8036\284865468.py:6: RuntimeWarning: divide by zero encountered in log
  preds = np.log(preds) / temperature


ля дитина…
аж почуть. і в світі! —лонята[12]
чумакась ствадили в коні
шукав заміялись
настуся дітись,
а то в крітняй, сердешная в рвесе
безталяннька завиваває,
сотник
і, крукечо-тихолю,
з нальбу домосе
старий два шевченка, без живати.
єйсуримівника сонечка
і хату-нах-виховали,
і петра. кущо зознає
вельною україну
бо вам дитину. була здає;
у сідистягачна буже


In [13]:
text_gen(diversities=[1.2])


----- diversity: 1.2
----- Generaring with seed: "ають саму мене
у садочок гулять.
а хоч п"
ають саму мене
у садочок гулять.
а хоч погене чи не попл

C:\Users\voldo\AppData\Local\Temp\ipykernel_8036\284865468.py:6: RuntimeWarning: divide by zero encountered in log
  preds = np.log(preds) / temperature


абе?»
во очі завдні вранім люди,
з світе дояка побачитим
тури з вечісити.
навіїсі рає, а може мати.
уй скажений від микить,
і на улиці компїли.
первасі може, слова
на туметься мурі любов.
отак, друже, полішкато
святого божої з хородялами».
кагрі та й я собі в домоомоголла,
трощи сам була,
одурила на пумачища,
цариці
ва силкішні, встаєш, і ягось,
отож обий, що добре,
то і не молобок
